In [1]:
import os
import glob
import rawpy
import exifread
import cv2 as cv
import numpy as np
from GaussianPyramid import HDRPyramid
import matplotlib.pyplot as plt

In [2]:
def generate_tiles(image, tile_size, stride):
    if isinstance(tile_size, int):
        tile_size = (tile_size, tile_size)
    if isinstance(stride, int):
        stride = (stride, stride)
    tiles = []
    h, w = image.shape
    tile_size_h, tile_size_w = tile_size
    stride_h, stride_w = stride
    for i in range(0, h - tile_size_h + 1, stride_h):
        row = []
        for j in range(0, w - tile_size_w + 1, stride_w):
            tile = image[i : i + tile_size_h, j : j + tile_size_w]
            row.append(tile)
        tiles.append(row)
    tiles = np.array(tiles)
    return tiles

def isTypeInt(image):
    return image.dtype in [np.uint8, np.uint16, np.uint32, np.uint64, np.int8, np.int16, np.int32, np.int64, np.uint]

def downsample_bayer(image):
    R = image[0 :: 2, 0 ::2] # 红色通道，取偶数行和偶数列的像素
    G1 = image[0 :: 2, 1 :: 2] # 绿色通道1， 取偶数行和奇数列的像素
    G2 = image[1 :: 2, 0 :: 2] # 绿色通道2，取奇数行和偶数列的像素
    B = image[1 :: 2,  1 :: 2] # 蓝色通道，取奇数行和奇数列的像素
    if isTypeInt(image):
        return np.right_shift(R + G1 + G2 + B + 2, 2)
    else:
        return (R + G1 + G2 + B) * 0.25


def computer_tiles_distance(refTiles, altTiles, offsets):
    """
    [TODO] 理解并独立地实现这个功能
    """
    m, n, ts, _ = refTiles.shape
    h, w, _ = offsets.shape
    res = np.zeros(shape=(h, w), dtype=refTiles.dtype)
    for i in range(h):
        for j in range(w):
            # offset value
            off_i = offsets[i, j, 0]
            off_j = offsets[i, j, 1]
            # reference index
            ri = i * (ts // 2)
            rj = j * (ts // 2)
            di = ri + int(off_i + (0.5 if off_i >= 0 else -0.5))
            dj = rj + int(off_j + (0.5 if off_j >= 0 else -0.5))
            di = 0 if di < 0 else (m - 1 if di > m - 1 else di)
            dj = 0 if dj < 0 else (n - 1 if dj > n - 1 else dj)
            dst = 0
            for p in range(ts):
                for q in range(ts):
                    dst += np.abs(refTiles[ri, rj, p, q] - altTiles[di, dj, p, q])
            res[i, j] = dst   
    return res

In [3]:
refIndex = 0
rawBayers = []
burstPath = "/home/lianghao/Documents/Learning/ISP/Image/IPOL_hdrplus_gopro_raw_bursts/hand_iso1600/"
alignmen_params = {
				'mode': 'bayer',  # images are single image Bayer / Color Filter Arrays
				'tuning': {
					# WARNING: these parameters are defined fine-to-coarse!
					'factors': [1, 2, 4, 4],
					'tileSizes': [16, 16, 16, 8],
					'searchRadia': [1, 4, 4, 4],
					'distances': ['L1', 'L2', 'L2', 'L2'],
					'subpixels': [False, True, True, True]  # if you want to compute subpixel tile alignment at each pyramid level
				},
				# rawpy parameters for images with motion fields
				'rawpyArgs': {
					'demosaic_algorithm' : rawpy.DemosaicAlgorithm.AHD,  # used in HDR+ supplement
					'half_size' : False,
					'use_camera_wb' : True,
					'use_auto_wb' : False,
					'no_auto_bright': True,
					'output_color' : rawpy.ColorSpace.sRGB,  # sRGB
					'output_bps' : 8
				},
				'writeMotionFields': False,
				'writeAlignedTiles': False
			}

rawPathList = glob.glob(os.path.join(burstPath, "*.dng"))
rawPathList.sort()
for rawPath in rawPathList:
    with rawpy.imread(rawPath) as rawObject:
        rawBayers.append(rawObject.raw_image.copy())

In [4]:
with rawpy.imread(rawPathList[refIndex]) as refRawpy:
    blackLevel = refRawpy.black_level_per_channel.copy()
    whiteLevel = refRawpy.white_level

In [5]:
h, w = rawBayers[refIndex].shape
if alignmen_params["mode"] == "bayer":
    tileSize = 2 * alignmen_params['tuning']['tileSizes'][0]
else:
    tileSize = alignmen_params['tuning']['tileSizes'][0]

In [6]:
# padding setting 
padding_height = (tileSize - h % tileSize) % tileSize
padding_width = (tileSize - w % tileSize) % tileSize
padding_overlap_height, padding_overlap_width = tileSize // 2, tileSize // 2
padding_top = padding_overlap_height
padding_bottom = padding_overlap_height + padding_height
padding_left = padding_overlap_width
padding_right = padding_overlap_width + padding_left

images_padding = []
for i in range(len(rawBayers)):
    im = np.pad(rawBayers[i], ((padding_top, padding_bottom), (padding_left, padding_right)), mode="symmetric")
    images_padding.append(im)

reference = images_padding[refIndex]
alternatives = [images_padding[i] for i in range(len(images_padding)) if i != refIndex]

implements the coarse-to-fine alignment on 4-level gaussian pyramids as defined in Algorithm 1 of Section 3 of the IPOL article.

In [7]:
# factors, tileSizes, distances, searchRadia and subpixels are described fine-to-coarse
factors = [1, 2, 4, 4]
tile_sizes =  [16, 16, 16, 8]
search_radia = [1, 4, 4, 4]
distances = ['L1', 'L2', 'L2', 'L2']
subpixels = [False, True, True, True]

upsampling_factors = factors[1 : ] + [1]
previous_tile_sizes = tile_sizes[1:] + [None]

# If dealing with raw images, 2x2 bayer pixels block are averaged
# (motion can then only be multiples of 2 pixels in original image size)
# 对参考帧进行下采样2倍
imRef = downsample_bayer(reference)
tile_size = 2 * tile_sizes[0]
refTiles = generate_tiles(reference, tile_size, tile_size // 2)

In [8]:
# 存储对齐瓦片
aligned_tiles = np.empty(((len(images_padding),) + refTiles.shape), dtype=refTiles.dtype) 
aligned_tiles[0] = refTiles # 参考帧的瓦片作为对齐的初始值
# 备选帧的瓦片对参考帧的瓦片的移动向量
motion_vectors = np.empty((len(alternatives), refTiles.shape[0], refTiles.shape[1], 2), dtype=int)
# 构建参考帧的4层从粗corse到细fine的金字塔
hdrPyramid = HDRPyramid()
refPyramid = hdrPyramid.gaussian_pyramid(image=imRef, factors=factors, ksize=5)

In [9]:
# 将每个备选帧对齐到参考帧
for i, alternateImg in enumerate(alternatives):
    # downsample bayer raw to grayscale
    im_alter = downsample_bayer(alternateImg)
    # 生成备选帧的4层由粗到细的金字塔
    alternatePyramid = hdrPyramid.gaussian_pyramid(im_alter, factors)
    # 金字塔由粗到细渐进式地对齐
    aligments = None
    for lv in range(len(refPyramid)):
        pass

金字塔对齐

1. 对之前对齐的offset进行上采样

In [ ]:
def upsample_alignments(refer_pyramid, alter_pyramid, pre_alignments, up_factor, tile_size, pre_tile_size):
    """
    [TODO] : 理解并掌握该函数的原理和实现方法
    """
    # 对金字塔上一层估计的偏移量进行上采样
    pre_alignments = pre_alignments * up_factor
    # 不同分辨率上采样因子和瓦片大小导致不同的向量重复率
    repeat_factor = upsampling_factors // (tile_size // pre_tile_size)
    upsample_alignments = pre_alignments.repeat(repeat_factor, 0).repeat(repeat_factor, 1)
    h, w = upsample_alignments.shape[:2]
    # 镜像填充mirror避免边缘效应
    pad_pre_alignments = np.pad(pre_alignments, pad_width=((1, ), (1, ), (0, )), mode='edge')
    # 创建一个最近邻居的掩码，指定使用的偏移量以获得正确的索引
    tile = np.zeros((repeat_factor, repeat_factor, 2, 2), dtype=np.int32)
    tile[:(repeat_factor // 2), :(repeat_factor // 2)] = [[-1, 0], [0, -1]] # 左上
    tile[:(repeat_factor // 2), (repeat_factor // 2) : ] = [[-1, 0], [0, 1]] # 右上
    tile[:(repeat_factor // 2):, :(repeat_factor // 2)] = [[1, 0], [0, -1]] # 左下
    tile[(repeat_factor // 2):, (repeat_factor // 2) : ] = [[1, 0], [0, 1]] # 右下
    
    neighborMask = np.tile(tile, (upsample_alignments.shape[0] // repeat_factor, upsample_alignments.shape[1] // repeat_factor, 1, 1))
    # Compute the indices of the neighbors using the offsets mask
    ti1 = np.repeat(np.clip(2 + np.arange(h) // repeat_factor + neighborMask[:, 0, 0, 0], 0, pad_pre_alignments.shape[0] - 1).reshape(h, 1), w, axis=1).reshape(h * w)
    ti2 = np.repeat(np.clip(2 + np.arange(h) // repeat_factor + neighborMask[:, 0, 1, 0], 0, pad_pre_alignments.shape[0] - 1).reshape(h, 1), w, axis=1).reshape(h * w)
    tj1 = np.repeat(np.clip(2 + np.arange(w) // repeat_factor + neighborMask[0, :, 0, 1], 0, pad_pre_alignments.shape[1] - 1).reshape(1, w), h, axis=0).reshape(h * w)
    tj2 = np.repeat(np.clip(2 + np.arange(w) // repeat_factor + neighborMask[0, :, 1, 1], 0, pad_pre_alignments.shape[1] - 1).reshape(1, w), h, axis=0).reshape(h * w)
    # Extract the previously estimated motion vectors associeted with those neighbors
    ppa1 = pad_pre_alignments[ti1, tj1].reshape((h, w, 2))
    ppa2 = pad_pre_alignments[ti2, tj2].reshape((h, w, 2))
    # 获得参考帧和备选帧所有可能的瓦片
    refTiles = generate_tiles(refer_pyramid, tile_size, 1)
    altTiles = generate_tiles(alter_pyramid, tile_size, 1)
    # 计算瓦片之间的距离
    d0 = computer_tiles_distance(refTiles, altTiles, upsample_alignments).reshape(h * w)
    d1 = computer_tiles_distance(refTiles, altTiles, ppa1).reshape(h * w)
    d2 = computer_tiles_distance(refTiles, altTiles, ppa2).reshape(h * w)
    # 获得候选的运动向量 : 上采样的偏移量、估计的偏移量
    candidate_alignments = np.empty((h * w, 3, 2))
    candidate_alignments[:, 0, :] = upsample_alignments.reshape(h * w, 2)
    candidate_alignments[:, 1, :] = ppa1.reshape(h * w, 2)
    candidate_alignments[:, 2, :] = ppa2.reshape(h * w, 2)
    # 通过最小化与参考瓦片的L1距离来选择最佳的偏移量
    select_alignments = candidate_alignments[np.arange(h * w), np.argmin([d0, d1, d2], axis=0)].reshape((h, w, 2))
    if h < refer_pyramid.shape[0] // (tile_size // 2) - 1 or w < refer_pyramid.shape[1] // ( tile_size // 2) - 1:
        new_alignments = np.zeros((refer_pyramid.shape[0] // (tile_size // 2) - 1, refer_pyramid.shape[1] // (tile_size // 2) - 1,))
        new_alignments[:h, :w] = select_alignments
    else:
        new_alignments = select_alignments
    
    return new_alignments

2. 对齐当前的金字塔层

In [1]:
def align_pyramid(refer_pyramid, alter_pyramid, up_factor, 
                  tile_size, pre_tile_size, search_radia, measure="L1", subpixel=True, pre_alignments=None):
    if isTypeInt(refer_pyramid):
        refer_pyramid = refer_pyramid.astype(np.float32)
    if isTypeInt(alter_pyramid):
        alter_pyramid = alter_pyramid.astype(np.float32)
    # 提取参考帧中的overlap-tiles
    refTiles = generate_tiles(refer_pyramid, tile_size, tile_size // 2)
    # 使用上采样之前的对齐位移量作为初始guess
    if pre_alignments is None:
        upsampled_alignments = np.zeros((refTiles.shape[0], refTiles.shape[1], 2), dtype=np.float32)
    else:
        upsampled_alignments = upsample_alignments(refer_pyramid, alter_pyramid, 
                                                   pre_alignments, up_factor, tile_size, pre_tile_size)
    
    h, w = upsampled_alignments.shape[:2]
    search_window = 2 * search_radia + 1
    
    u0 = np.round(upsampled_alignments[:, :, 0]).astype(np.int32)
    v0 = np.round(upsampled_alignments[:, :, 1]).astype(np.int32)
    
    search_areas = generate_tiles(np.pad(alter_pyramid, search_radia, mode="constant", constant_values=2 **16 - 1),
                                  tile_size= tile_size + 2 * search_radia)
    
    # Only keep those corresponding to the area around the reference tile location + [u0, v0]
    indI = np.clip(((np.repeat((np.arange(h) * tile_size // 2).reshape(h, 1), w, axis=1)).reshape(h, w) + u0).reshape(h * w), 0, search_areas.shape[0] - 1)
    indJ = np.clip(((np.repeat((np.arange(w) * tile_size // 2).reshape(1, w), h, axis=0)).reshape(h, w) + v0).reshape(h * w), 0, search_areas.shape[1] - 1)
	